# ZnO Slab Construction with MolPy

This notebook demonstrates how to construct a ZnO surface slab using the MolPy framework. The workflow includes:

1. **Reading** a ZnO wurtzite structure from an XSF file
2. **Creating a supercell** to build a 4×2×3 slab structure
3. **Adding vacuum space** to prevent periodic image interactions
4. **Visualizing** the resulting structure
5. **Writing** the final structure to a file

This workflow is commonly used in surface science and computational materials research to prepare realistic surface models for further calculations.

## 1. Import Required Libraries

First, we'll import all the necessary modules from MolPy for structure manipulation and visualization.

In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
from pathlib import Path

data_path = Path("data/case3/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Read ZnO Wurtzite Structure

We'll read the ZnO wurtzite unit cell from an XSF (XCrySDen Structure Format) file. This format is commonly used for crystal structures.

**Note**: For this demo, we'll create a sample ZnO wurtzite structure if the file doesn't exist.

In [7]:
ZnO_surf = mp.io.read_xsf(data_path / "ZnO_wurtzite.xsf")

## 3. Create Supercell Slab

Now we'll replicate the unit cell to create a slab structure. The transformation matrix `[[4,0,0],[0,2,0],[0,0,3]]` means:
- **4× replication** in the x-direction
- **2× replication** in the y-direction  
- **3× replication** in the z-direction (thickness of 3 layers)

This creates a 4×2×3 supercell that's suitable for surface calculations.

In [8]:
# Create PeriodicSystem wrapper for supercell operations
ZnO_periodic = mp.PeriodicSystem(ZnO_surf)

# Create the 4x2x3 supercell using the make_supercell method
transformation_matrix = [[4, 0, 0], [0, 2, 0], [0, 0, 3]]
ZnO_surf_sc = ZnO_periodic.make_supercell(transformation_matrix)

print("Supercell created successfully!")
print(f"Original structure: {len(ZnO_surf._wrapped['atoms']['xyz'])} atoms")
print(f"Supercell structure: {len(ZnO_surf_sc._wrapped._wrapped['atoms']['xyz'])} atoms")
print(f"Replication factor: {len(ZnO_surf_sc._wrapped._wrapped['atoms']['xyz']) // len(ZnO_surf._wrapped['atoms']['xyz'])}×")

print(f"\\nBox dimensions:")
print(f"Original: {ZnO_surf.box.lengths}")
print(f"Supercell: {ZnO_surf_sc.box.lengths}")
print(f"Scale factors: {ZnO_surf_sc.box.lengths / ZnO_surf.box.lengths}")

Supercell created successfully!
Original structure: 8 atoms
Supercell structure: 192 atoms
Replication factor: 24×
\nBox dimensions:
Original: [3.282      5.291      5.68459075]
Supercell: [13.128      10.582      17.05377225]
Scale factors: [4. 2. 3.]


## 4. Add Vacuum Layer

Now we'll add a vacuum layer of 25.0 Angstroms in the z-direction to create a proper surface slab. This vacuum prevents periodic images from interacting across the surface normal direction.

In [9]:
# Add 25.0 Angstrom vacuum in the z-direction
zno_surf_vacuum = ZnO_surf_sc.add_vacuum(25.0, direction='z')

print(f"Original supercell box dimensions: {ZnO_surf_sc.box.matrix.diagonal()}")
print(f"With vacuum box dimensions: {zno_surf_vacuum.box.matrix.diagonal()}")
print(f"Vacuum added: {zno_surf_vacuum.box.matrix[2,2] - ZnO_surf_sc.box.matrix[2,2]:.2f} Angstrom")

Original supercell box dimensions: [13.128      10.582      17.05377225]
With vacuum box dimensions: [13.128      10.582      42.05377225]
Vacuum added: 25.00 Angstrom


## 5. Visualize the ZnO Slab

Let's visualize the final ZnO slab structure to verify our construction. This will help us confirm the supercell was created correctly and the vacuum layer was added properly.

In [5]:
# Visualize the ZnO slab
# Note: This assumes molpy has a visualization function
# If not available, you can use other visualization tools like ASE or matplotlib

try:
    # Try to use molpy's view function if available
    from molpy.view import view
    view(zno_surf_vacuum)
except ImportError:
    print("Visualization function not available in molpy")
    print("Alternative: Use ASE or other visualization tools")
    print("Structure information:")
    print(f"- Total atoms: {len(zno_surf_vacuum._wrapped._wrapped['atoms']['xyz'])}")
    print(f"- Box dimensions: {zno_surf_vacuum.box.matrix.diagonal()}")
    print(f"- Vacuum in z-direction: {zno_surf_vacuum.box.matrix[2,2] - zno_surf_sc.box.matrix[2,2]:.2f} Angstrom")

Visualization function not available in molpy
Alternative: Use ASE or other visualization tools
Structure information:
- Total atoms: 192
- Box dimensions: [13.128      10.582      42.05377225]


NameError: name 'zno_surf_sc' is not defined

## 6. Write the ZnO Slab to File

Finally, we'll write the constructed ZnO slab to an XSF file for future use or analysis with other software.

In [ ]:
# Write the ZnO slab to XSF file
output_filename = 'ZnO_wurtzite_4x2x3.xsf'

# Get the underlying system for writing
zno_surf_vacuum["atoms"]["type"] = zno_surf_vacuum["atoms"]["atomic_number"]

# Write using molpy's write function
mp.io.write_lammps_data(output_filename, zno_surf_vacuum)

print(f"ZnO slab written to {output_filename}")

KeyError: 'mass'

## Summary and Notes

This notebook demonstrated the complete workflow for constructing a ZnO surface slab using molpy:

1. **Import Libraries**: Imported molpy and its I/O functions
2. **Read Structure**: Loaded the ZnO wurtzite unit cell from an XSF file
3. **Create Supercell**: Used the `PeriodicSystem.make_supercell()` method to create a 4x2x3 supercell
4. **Add Vacuum**: Used the `PeriodicSystem.add_vacuum()` method to add 25 Å vacuum in the z-direction
5. **Visualize**: Displayed the final structure (when visualization tools are available)
6. **Write File**: Saved the final slab structure to an XSF file

### Key Features of molpy's PeriodicSystem:

- **Robust supercell creation**: Handles atomic replication and property preservation
- **Vacuum addition**: Expands simulation box without moving atoms
- **Type safety**: Only works with periodic systems
- **Property preservation**: Maintains all atomic properties through transformations

### Next Steps:

- Use the generated slab for surface calculations
- Add adsorbates to study surface chemistry
- Perform relaxation or dynamics simulations
- Analyze surface properties